In [244]:
import requests 
import pandas as pd
import numpy as np
website_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
table= soup.find('table',class_='wikitable sortable')
a=table.findAll('tr')


In [2]:
list(list(a)[2])[1]

Borough=[]
Postalcode=[]
Neighbourhood=[]
for i in range(1,len(a)):
    Borough.append(list(list(a)[i])[3])
    Postalcode.append(list(list(a)[i])[1])
    Neighbourhood.append(list(list(a)[i])[5])

In [4]:
#clean Postalcode
for i in range(0,len(Postalcode)):
    Postalcode[i]=str(Postalcode[i]).replace("<td>", "")
    Postalcode[i]=str(Postalcode[i]).replace("</td>", "")
    Postalcode[i]=str(Postalcode[i]).replace("\n", "")

Postalcode
for i in range(0,len(Neighbourhood)):
    Neighbourhood[i]=str(Neighbourhood[i]).replace("<td>", "")
    Neighbourhood[i]=str(Neighbourhood[i]).replace("</td>", "")
    Neighbourhood[i]=str(Neighbourhood[i]).replace("\n", "")

for i in range(0,len(Neighbourhood)):
    Borough[i]=str(Borough[i]).replace("<td>", "")
    Borough[i]=str(Borough[i]).replace("</td>", "")
    Borough[i]=str(Borough[i]).replace("\n", "")

 

In [5]:

borough_neighborhoud_names_shearch=table.find_all('a')
len(borough_neighborhoud_names_shearch)
borough_neighborhoud=[]
for i  in range(0, len(borough_neighborhoud_names_shearch)):
    borough_neighborhoud.append(borough_neighborhoud_names_shearch[i].text)
borough_neighborhoud

count=0
for i in range(0,len(Borough)):
    if(Borough[i].startswith('<a')):
        Borough[i]=borough_neighborhoud[count]
        count=count+1
        
    if(Neighbourhood[i].startswith('<a')):
        Neighbourhood[i]=borough_neighborhoud[count]
        count=count+1

data={'PostalCode':Postalcode,
      'Borough':Borough,
      'Neighbourhood':Neighbourhood}
tabela_final=pd.DataFrame(data)
#we only want borough assigned
tabela_final=tabela_final.loc[tabela_final['Borough'] != 'Not assigned']
tabela_final
for i in range(0,len(tabela_final)):
    if(tabela_final.iloc[i,2]=='Not assigned'):
        tabela_final.iloc[i,2]=tabela_final.iloc[i,1]
    
tabela_final

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


In [6]:
new_Postalcode=[]
new_Borough=[]
new_Neighbourhood=[]

count=0
x=0
for i in range(0,len(tabela_final)):
    if(i!=0):
        x=x+count
    count=0  
    if(x<len(tabela_final)):  
        for j in range(0,len(tabela_final)):    
            if(tabela_final.iloc[x,0]==tabela_final.iloc[j,0]):
                count=count+1
                if(count==1):
                    new_Postalcode.append(tabela_final.iloc[x,0])
                    new_Borough.append(tabela_final.iloc[x,1])
                    new_Neighbourhood.append(tabela_final.iloc[x,2])
                elif(count>1):
                    
                    #add coma
                    #print(tabela_final.iloc[i,0],j,'j')
                    #print(tabela_final.iloc[j,0],i,'i')
                    var=', '
                    new_Neighbourhood[-1]=new_Neighbourhood[-1]+var+tabela_final.iloc[j,2]
    
new_Neighbourhood
new_data={'PostalCode':new_Postalcode,
      'Borough':new_Borough,
      'Neighbourhood':new_Neighbourhood}
new_tabela_final=pd.DataFrame(new_data)
new_tabela_final



,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


In [7]:
df = pd.read_csv('http://cocl.us/Geospatial_data')
df.head
log_vector=np.zeros(len(new_Postalcode))
lat_vector=np.zeros(len(new_Postalcode))
for i in range(0,len(log_vector)):
    for j in range(0,len(log_vector)):
        if(new_Postalcode[i]==df['Postal Code'][j]):
            lat_vector[i]=df['Latitude'][j]
            log_vector[i]=df['Longitude'][j]



In [8]:
new_data={'PostalCode':new_Postalcode,
      'Borough':new_Borough,
      'Neighbourhood':new_Neighbourhood,
      'Latitude':lat_vector,
      'Longitude':log_vector}
new_tabela_final=pd.DataFrame(new_data)
new_tabela_final

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509


In [ ]:
#Now let's start our folium map with all Borough's
import folium

In [34]:
#! pip install geocoder  I needed to install geocoder packages
adress='Toronto, ON, Canada'
a = geocoder.osm(adress)
latitude=a.osm['y']
longitude=a.osm['x']

print('Coordinates from Toronto: latitude {}, longitude {},'.format(latitude,longitude))
#latitude = lat_lng_coords[0]


Coordinates from Toronto latitude 43.653963, longitude -79.387207,


In [52]:
#now we're going to create a folium map with all boroughs'
Toronto_city = folium.Map(location=[latitude, longitude], zoom_start=10)

for i in range(0,len(new_tabela_final)):
    label = '{}, {}'.format(new_tabela_final.iloc[i][2], new_tabela_final.iloc[i][1])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [new_tabela_final.iloc[i]['Latitude'], new_tabela_final.iloc[i]['Longitude']],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#2186cc',
        fill_opacity=0.2,
        parse_html=False).add_to(Toronto_city)

Toronto_city
    




In [53]:
#Now we will connect to Foursquare API

CLIENT_ID = 'JNPZF4KENJVWLOTSWMDSLX1DK3LGT502M3S30K1JBSHIWKTY' # your Foursquare ID
CLIENT_SECRET = '0R4QIE0JXQ4KS1EQQDDMKS1WP3BQIINMRNWUCU0LBZ1T1FN1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: JNPZF4KENJVWLOTSWMDSLX1DK3LGT502M3S30K1JBSHIWKTY
CLIENT_SECRET:0R4QIE0JXQ4KS1EQQDDMKS1WP3BQIINMRNWUCU0LBZ1T1FN1


# We only going to study Borought that have word Toronto. First of all, we need to split or dataset 


In [73]:
DF_toronto=[]
for i in range(0,len(new_tabela_final)):
    if('Toronto' in new_tabela_final['Borough'][i]):
           DF_toronto.append(new_tabela_final.iloc[i])


DF_toronto=pd.DataFrame(DF_toronto)
DF_toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Now, let's get the top 100 venues that are in Harbourfront within a radius of 500 meters.


In [94]:
#coordinates from Harbourfront
coords=DF_toronto.query('Neighbourhood=="Harbourfront"').iloc[0][['Latitude','Longitude']]
coords

Latitude     43.6543
Longitude   -79.3606
Name: 2, dtype: object

In [96]:
#our query to search at foursquare
radius = 500
LIMIT=100
neighborhood_latitude=coords[0]
neighborhood_longitude=coords[1]
VERSION = '20180604'
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)

In [97]:
#Let's get the results
results = requests.get(url).json()


In [105]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
#Now we are ready to clean the json and structure it into a pandas dataframe.

In [111]:
venues = results['response']['groups'][0]['items']
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [113]:
# now we will reproduce all process into a function

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            print(name)
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
            # return only relevant information for each nearby venue
            venues_list.append([(
                name,
                lat,
                lng,
                v['venue']['name'],
                v['venue']['location']['lat'],
                v['venue']['location']['lng'],
                v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude',
                  'Neighborhood Longitude',
                  'Venue',
                  'Venue Latitude',
                  'Venue Longitude',
                  'Venue Category']
    return(nearby_venues)

In [119]:
toronto_venues = getNearbyVenues(names=DF_toronto['Neighbourhood'],
                                 latitudes=DF_toronto['Latitude'],
                                 longitudes=DF_toronto['Longitude'])

Harbourfront
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian P

In [167]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [122]:
#now let's check our result, it's possible to check how many places we get per each neighborhood
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15


In [123]:
#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


# 3. Analyze Each Neighborhood

In [168]:
# one hot encoding
toronto_onehot = 1
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
#add neighborhood column back to dataframe
toronto_onehot.insert(0,'Neighborhoud',toronto_venues['Neighborhood'])



In [172]:
toronto_onehot.head()

,Neighborhoud,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [173]:
toronto_onehot.shape

(1686, 236)

In [174]:
toronto_grouped = toronto_onehot.groupby('Neighborhoud').mean().reset_index()

In [175]:
toronto_grouped

,Neighborhoud,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,0.000000,...,0.000000,0.000000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.000000,0.012195
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.021739,0.000000,0.043478,0.010870,0.000000,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.011905,0.000000,0.000000,0.011905,0.011905


# Let's print each neighborhood along with the top 5 most common venues

In [178]:
num_top_venues = 5
for hood in toronto_grouped['Neighborhoud']:
        print("----"+hood+"----")
        temp = toronto_grouped[toronto_grouped['Neighborhoud'] == hood].T.reset_index()
        temp.columns = ['venue','freq']
        temp = temp.iloc[1:]
        temp['freq'] = temp['freq'].astype(float)
        temp = temp.round({'freq': 2})
        print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
        print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1          Bakery  0.05
2  Farmers Market  0.04
3    Cocktail Bar  0.04
4        Beer Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
                   venue  freq
0         Breakfast Spot  0.09
1                   Café  0.09
2  Performing Arts Venue  0.09
3                 Bakery  0.09
4            Coffee Shop  0.09


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2                Park  0.05
3          Comic Shop  0.05
4    Recording Studio  0.05


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0      Airport Lounge  0.13
1     Air

# Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [221]:
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]



In [228]:
neighborhoods_venues_sorted=None
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhoud']

for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))
            
            
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoud'] = toronto_grouped['Neighborhoud']

for ind in np.arange(toronto_grouped.shape[0]):
        neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhoud,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,Cosmetics Shop,Asian Restaurant,Restaurant,Burger Joint,American Restaurant
1,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Beer Bar,Café,Steakhouse,Farmers Market,Seafood Restaurant,Breakfast Spot
2,"Brockton, Exhibition Place, Parkdale Village",Performing Arts Venue,Coffee Shop,Café,Breakfast Spot,Bakery,Intersection,Pet Store,Climbing Gym,Restaurant,Sandwich Place
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boutique,Plane,Boat or Ferry


# 4. Cluster Neighborhoods

In [229]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhoud', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe

kmeans.labels_[0:10]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [230]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = DF_toronto
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoud'), on='Neighbourhood')
toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Mexican Restaurant,Theater,Café,Hotel,Gym / Fitness Center
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,3,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Bakery,Fast Food Restaurant,Restaurant,Japanese Restaurant,Tea Room
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Café,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Clothing Store,Diner,Cosmetics Shop,Beer Bar,Bakery
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Other Great Outdoors,Trail,Neighborhood,Pizza Place,Health Food Store,Event Space,Ethiopian Restaurant,Electronics Store,Dim Sum Restaurant
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Beer Bar,Café,Steakhouse,Farmers Market,Seafood Restaurant,Breakfast Spot


# Finally, let's visualize the resulting clusters

In [243]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
                label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
                folium.CircleMarker(
                [lat, lon],
                radius=5,
                popup=label,
                color=rainbow[cluster-1],
                fill=True,
                fill_color=rainbow[cluster-1],
                fill_opacity=0.7).add_to(map_clusters)
map_clusters

# 5. Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

In [236]:
# 1st cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,0,Pub,Other Great Outdoors,Trail,Neighborhood,Pizza Place,Health Food Store,Event Space,Ethiopian Restaurant,Electronics Store,Dim Sum Restaurant
67,Central Toronto,0,Food & Drink Shop,Park,Asian Restaurant,Clothing Store,Hotel,Sandwich Place,Gym,Breakfast Spot,Event Space,Ethiopian Restaurant
68,Central Toronto,0,Mexican Restaurant,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
83,Central Toronto,0,Intersection,Gym,Tennis Court,Trail,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Diner,Electronics Store


In [237]:
# 2nd cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,1,Garden,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [238]:
# 3rd cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [239]:
# 4th cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Mexican Restaurant,Theater,Café,Hotel,Gym / Fitness Center
9,Downtown Toronto,3,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Bakery,Fast Food Restaurant,Restaurant,Japanese Restaurant,Tea Room
15,Downtown Toronto,3,Café,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Clothing Store,Diner,Cosmetics Shop,Beer Bar,Bakery
20,Downtown Toronto,3,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Beer Bar,Café,Steakhouse,Farmers Market,Seafood Restaurant,Breakfast Spot
24,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Ice Cream Shop,Bakery,Dessert Shop,Bar,Spa
25,Downtown Toronto,3,Grocery Store,Café,Park,Restaurant,Convenience Store,Diner,Baby Store,Italian Restaurant,Coffee Shop,Candy Store
30,Downtown Toronto,3,Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,Cosmetics Shop,Asian Restaurant,Restaurant,Burger Joint,American Restaurant
31,West Toronto,3,Bakery,Pharmacy,Supermarket,Park,Middle Eastern Restaurant,Bank,Music Venue,Bar,Café,Brewery
36,Downtown Toronto,3,Coffee Shop,Hotel,Aquarium,Café,Brewery,Scenic Lookout,Italian Restaurant,Fried Chicken Joint,Restaurant,Park
37,West Toronto,3,Bar,Coffee Shop,Men's Store,Asian Restaurant,Restaurant,Vietnamese Restaurant,Café,Pizza Place,Bakery,New American Restaurant


In [240]:
# 5th cluster
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Downtown Toronto,4,Park,Playground,Trail,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
